# RandomForestClassifier 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV


# Leer el DataFrame desde un archivo CSV
df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

# Características (features) del modelo
X = df_historico.drop('Deporte', axis=1)
y = df_historico['Deporte']

# Dividir los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=0)



parameters = {"n_estimators":[50,100,150],
              "max_depth":[2,3,4],
              "criterion": ['gini', 'entropy'],
              "min_samples_split": [5,8],
              "min_samples_leaf": [2,3],
              "max_features": [1,2]
}

dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="accuracy", )


dtr_gs.fit(X_train, y_train)

dtr_gs.best_estimator_

y_pred = dtr_gs.best_estimator_.predict(X_test)

print("accuracy_score", accuracy_score(y_test, y_pred))
print("recall_score", recall_score(y_test, y_pred))
print("precision_score", precision_score(y_test, y_pred))
print("roc_auc_score", roc_auc_score(y_test, y_pred))

accuracy_score 0.48
recall_score 0.2909090909090909
precision_score 0.5517241379310345
roc_auc_score 0.5010101010101011


In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# Resto del código...

# Aplicar SMOTE para sobre-muestrear las clases minoritarias
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Crear el modelo RandomForestClassifier
modelo = RandomForestClassifier(random_state=42)

# Definir la rejilla de hiperparámetros a buscar
param_grid = {
    'n_estimators': [50, 100, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Realizar la búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid,)
grid_search.fit(X_resampled, y_resampled)

# Obtener los mejores hiperparámetros encontrados
mejores_hiperparametros = grid_search.best_params_
print("Mejores hiperparámetros:", mejores_hiperparametros)

# Entrenar el modelo con los mejores hiperparámetros en todo el conjunto de entrenamiento
modelo_optimizado = RandomForestClassifier(**mejores_hiperparametros, random_state=42)
modelo_optimizado.fit(X_resampled, y_resampled)

# Realizar predicciones en el conjunto de prueba con el modelo optimizado
predicciones = modelo_optimizado.predict(X_test)

# Calcular la precisión del modelo en el conjunto de prueba
precision = accuracy_score(y_test, predicciones)
print("Precisión del modelo en el conjunto de prueba:", precision)

Mejores hiperparámetros: {'max_depth': 10, 'min_samples_split': 10, 'n_estimators': 100}
Precisión del modelo en el conjunto de prueba: 0.515


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score,recall_score,precision_score

# Leer el DataFrame desde un archivo CSV
df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

# Características (features) del modelo
X = df_historico.drop('Deporte', axis=1)
# Etiqueta del modelo: Deporte practicado
y = df_historico['Deporte']

# Dividir los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
modelo = RandomForestClassifier(random_state=42)
modelo.fit(X_train, y_train)

# Realizar la predicción del modelo con los datos de prueba
predicciones = modelo.predict(X_test)

precision = accuracy_score(y_test, predicciones)
print("Precisión del modelo:", precision)


print("accuracy_score", accuracy_score(y_test, predicciones))
# print("recall_score", recall_score(y_test, predicciones))
print("precision_score", precision_score(y_test, predicciones))
print("roc_auc_score", roc_auc_score(y_test, predicciones))


recall_macro = recall_score(y_test, predicciones, average='macro')
print("Recall promedio (macro):", recall_macro)



Precisión del modelo: 0.525
accuracy_score 0.525
precision_score 0.5842696629213483
roc_auc_score 0.5308080808080808
Recall promedio (macro): 0.5308080808080808


In [4]:
import pickle

import os

folder = "modelos_en_pkl"

# Asegúrate de que la carpeta exista. Si no existe, créala.
if not os.path.exists(folder):
    os.makedirs(folder)

# Ruta completa del archivo donde se guardará el modelo
filename = os.path.join(folder, "Randomforest.pkl")

# Guardar el modelo en el archivo
with open(filename, 'wb') as file:
    pickle.dump(model, file)

#  RandomForestClassifier Con datos de el clima..   NO VALE

In [33]:
import json
import csv
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Función para leer datos meteorológicos desde un archivo JSON
def leer_datos_meteorologicos(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        datos = json.load(archivo)
    return datos

# Función para etiquetar la presencia de una ola de calor basada en datos climáticos
def etiquetar_ola_de_calor(datos_clima, umbral_temperatura):
    etiquetas = []
    for registro in datos_clima['data']['list']:
        temperatura = registro['main']['temp']
        if temperatura > umbral_temperatura:
            etiquetas.append(1)  # Hay ola de calor
        else:
            etiquetas.append(0)  # No hay ola de calor
    return etiquetas

# Función para cargar datos de instalaciones desde un archivo CSV y filtrar por tipo de instalación
def cargar_datos_instalaciones(nombre_archivo_csv, tipo_instalacion):
    csv.field_size_limit(500 * 1024)
    datos_instalaciones = []
    with open(nombre_archivo_csv, 'r', encoding='utf-8', newline='') as archivo_csv:
        lector_csv = csv.DictReader(archivo_csv)
        for fila in lector_csv:
            if fila['TIPO'] == tipo_instalacion:
                datos_instalaciones.append(fila)
    return datos_instalaciones

# Función para obtener recomendaciones de deportes utilizando el modelo de machine learning
def obtener_recomendaciones_deportes_modelo(datos_clima, etiquetas_ola_calor, datos_instalaciones):
    # Obtener los datos históricos de recomendaciones de deportes
    # Aquí asumimos que tienes un archivo CSV llamado 'datos_historicos_deportes.csv'
    # que contiene datos históricos con columnas como 'Temperatura', 'OlaDeCalor', 'TipoInstalacion' y 'Recomendacion'
    df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

    # Características (features) del modelo: Temperatura y Ola de calor (1 si hay ola de calor, 0 si no)
    X = df_historico.drop('Deporte', axis=1)

    # Etiquetas del modelo: Recomendación de deporte
    y = df_historico['Deporte'].values

    # Entrenar el modelo Random Forest
    modelo = RandomForestClassifier(random_state=42)
    modelo.fit(X, y)

    # Obtener los valores actuales de Temperatura y Ola de calor
    temperatura_actual = datos_clima['data']['list'][0]['main']['temp']
    ola_de_calor_actual = etiquetas_ola_calor[0]

    # Realizar la predicción del modelo con los datos actuales
    prediccion_actual = modelo.predict([[temperatura_actual, ola_de_calor_actual]])

    recomendaciones = []

    if prediccion_actual[0] == 1:
        # Si el modelo predice que hay que evitar deportes, entonces:
        # Recomendar actividades en centros de salud si tiene problemas de salud
        if tiene_problemas_salud:
            recomendaciones.extend(obtener_recomendaciones_centros_salud(datos_instalaciones))
    else:
        # Si el modelo predice que se pueden hacer deportes, entonces:
        # Recomendar actividades al aire libre en parques y jardines
        recomendaciones.extend(obtener_recomendaciones_parques(datos_instalaciones))

    return recomendaciones

# Función para obtener recomendaciones específicas para parques y jardines
def obtener_recomendaciones_parques(instalaciones):
    recomendaciones_parques = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'parque':
            recomendaciones_parques.append(f"Hacer ejercicio en el parque: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_parques

# Función para obtener recomendaciones específicas para centros de salud
def obtener_recomendaciones_centros_salud(instalaciones):
    recomendaciones_centros_salud = []
    for instalacion in instalaciones:
        if instalacion['TIPO'].lower() == 'centro_salud' and tiene_problemas_salud:
            recomendaciones_centros_salud.append(f"Realizar actividad en centro de salud: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_centros_salud

if __name__ == "__main__":
    nombre_archivo_json = "../Raw_Datasets/Trabajado/tiempo.json"  # Reemplaza con el nombre de tu archivo JSON
    nombre_archivo_sitios = "../Raw_Datasets/items_Giacomo/items.csv"  # Reemplaza con el nombre de tu archivo CSV de sitios (piscinas, parques y polideportivos)

    umbral_temperatura_ola_calor = 30.0  # Reemplaza con el umbral de temperatura para definir una ola de calor
    tiene_problemas_salud = True  # Reemplaza con True si el usuario tiene problemas de salud, False si no

    datos_clima = leer_datos_meteorologicos(nombre_archivo_json)
    etiquetas_ola_calor = etiquetar_ola_de_calor(datos_clima, umbral_temperatura_ola_calor)

    tipos_instalaciones_deportivas = {'PISCINA', 'ZONA VERDE', 'SENDA', 'CENTRO DE SALUD'}

    datos_instalaciones = []
    for tipo in tipos_instalaciones_deportivas:
        datos_instalaciones.extend(cargar_datos_instalaciones(nombre_archivo_sitios, tipo))

    # Obtener recomendaciones de deportes utilizando el modelo de machine learning
    recomendaciones_deportes_modelo = obtener_recomendaciones_deportes_modelo(
        datos_clima, etiquetas_ola_calor, datos_instalaciones
    )

    # Imprime las recomendaciones del modelo
    print("Recomendaciones de deportes (modelo):")
    for deporte in recomendaciones_deportes_modelo:
        print(deporte)


C:\Users\de969\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 2 features, but RandomForestClassifier is expecting 72 features as input.

# RadientBoostingClassifier

In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

# Leer el DataFrame desde un archivo CSV
df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

# Características (features) del modelo
X = df_historico.drop('Deporte', axis=1)
y = df_historico['Deporte']


# Dividir los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingClassifier(random_state=0)

parameters = {
    "n_estimators": [50, 100, 150],
    "learning_rate": [0.1, 0.05, 0.01],
    "max_depth": [3, 4, 5],
}

dtr_gs = GridSearchCV(model, parameters, cv=5, scoring="accuracy")
dtr_gs.fit(X_train, y_train)

y_pred = dtr_gs.best_estimator_.predict(X_test)

print("accuracy_score:", accuracy_score(y_test, y_pred))
print("recall_score:", recall_score(y_test, y_pred,))
print("precision_score:", precision_score(y_test, y_pred,))
print("roc_auc_score:", roc_auc_score(y_test, y_pred,))


accuracy_score: 0.495
recall_score: 0.43636363636363634
precision_score: 0.5517241379310345
roc_auc_score: 0.5015151515151515


In [35]:
import pickle

import os

# Supongamos que quieres guardar el modelo en la carpeta "modelos"
# dentro del directorio actual
folder = "../modelos_en_pkl"

# Asegúrate de que la carpeta exista. Si no existe, créala.
if not os.path.exists(folder):
    os.makedirs(folder)

# Ruta completa del archivo donde se guardará el modelo
filename = os.path.join(folder, "GradientBoosting.pkl")

# Guardar el modelo en el archivo
with open(filename, 'wb') as file:
    pickle.dump(model, file)


# SVC

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import roc_auc_score

# Leer el DataFrame desde un archivo CSV
df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

# Características (features) del modelo
X = df_historico.drop('Deporte', axis=1)
y = df_historico['Deporte']

# Dividir los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo SVM con kernel lineal
modelo = OneVsOneClassifier(SVC(kernel='linear', random_state=42))

# Entrenar el modelo
modelo.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)

# Calcular las métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred, average='macro')
# roc_auc = roc_auc_score(y_test, y_pred, multi_class='ovo')

# Imprimir las métricas de evaluación
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
# print("ROC AUC:", roc_auc)


In [ ]:
folder = "modelos_en_pkl"

# Asegúrate de que la carpeta exista. Si no existe, créala.
if not os.path.exists(folder):
    os.makedirs(folder)

# Ruta completa del archivo donde se guardará el modelo
filename = os.path.join(folder, "svc.pkl")

# Guardar el modelo en el archivo
with open(filename, 'wb') as file:
    pickle.dump(model, file)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score
from sklearn.multiclass import OneVsOneClassifier

# Leer el DataFrame desde un archivo CSV
df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

# Características (features) del modelo
X = df_historico.drop('Deporte', axis=1)
y = df_historico['Deporte']

# Dividir los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar las formas de los datos para asegurarse de que tengan la misma cantidad de características
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

# Crear el modelo SVM con kernel lineal y enfoque One-vs-One
modelo = SVC(kernel='linear', random_state=42)

# Entrenar el modelo
modelo.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)

# Calcular las métricas de evaluación
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred, average='macro')
roc_auc = roc_auc_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("ROC AUC:", roc_auc)


Forma de X_train: (800, 72)
Forma de X_test: (200, 72)
Forma de y_train: (800,)
Forma de y_test: (200,)


# LINEAR REGRESSION

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Leer el DataFrame desde un archivo CSV
df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

# Características (features) del modelo
X = df_historico.drop('Deporte', axis=1)
y = df_historico['Deporte']

# Dividir los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión lineal
modelo = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
modelo.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)


Mean Squared Error: 0.26916805875125077
R-squared: -0.08754771212626555


# ARBOL

In [3]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# Leer el DataFrame desde un archivo CSV
df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

# Características (features) del modelo
X = df_historico.drop('Deporte', axis=1)
y = df_historico['Deporte']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 72)
(200, 72)
(800,)
(200,)


In [12]:
model = DecisionTreeRegressor(random_state=0)
model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=0)

In [13]:
model.score(X_test, y_test)

-0.96

In [14]:
import numpy as np

In [15]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

y_pred = model.predict(X_test)

print("R2", r2_score(y_test, y_pred))
print("MAE", mean_absolute_error(y_test, y_pred))
print("MAPE", mean_absolute_percentage_error(y_test, y_pred))
print("MSE", mean_squared_error(y_test, y_pred))
print("RMSE", np.sqrt(mean_squared_error(y_test, y_pred)))

R2 -0.96
MAE 0.49
MAPE 923237923610952.0
MSE 0.49
RMSE 0.7


In [16]:
import pickle

import os

folder = "modelos_en_pkl"

# Asegúrate de que la carpeta exista. Si no existe, créala.
if not os.path.exists(folder):
    os.makedirs(folder)

# Ruta completa del archivo donde se guardará el modelo
filename = os.path.join(folder, "DecisionTree.pkl")

# Guardar el modelo en el archivo
with open(filename, 'wb') as file:
    pickle.dump(model, file)